In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/pattern/traditional_algorithms

/content/drive/My Drive/pattern/traditional_algorithms


In [ ]:
%ls

groundtruth/  org/       otsu_result/     sr_sauvola_result/
gt/           original/  sauvola_result/  Traditional_Alg_code.ipynb


# Requirements

In [3]:
import cv2
import os
import numpy as np
from tqdm.auto import tqdm
import re
import torch
from tabulate import tabulate
from torch import nn



# otsu

In [ ]:
def apply_otsu(src_uniform,otsu_images):

    # Read the image

    os.makedirs(otsu_images, exist_ok=True)
    for img in tqdm(os.listdir(src_uniform),desc="otsu progress"):
        image = cv2.imread(os.path.join(src_uniform,img))
        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Otsu's thresholding
        _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        # Save the image using cv2.imwrite
        count=extract_number_from_string(img)
        otsu_filename = f'otsu_image{count}.png'
        otsu_filepath = os.path.join(otsu_images, otsu_filename)

        cv2.imwrite(otsu_filepath, thresholded)



# niblac

# applying sauvola_binarization

In [ ]:
def sauvola_binarization_folder(src_folder, suv_images, window_size=25, k=0.2):


    os.makedirs(suv_images, exist_ok=True)
    count = 1

    for img in tqdm(os.listdir(src_folder),desc="sauvola binarization progess"):
        image = cv2.imread(os.path.join(src_folder, img))
        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Define a function for Sauvola thresholding
        def sauvola_threshold(local_mean, local_stddev):
            return local_mean * (1 + k * ((local_stddev / 128) - 1))

        # Apply Sauvola thresholding to each pixel in the image
        result_image = np.zeros_like(gray_image, dtype=np.uint8)
        pad = window_size // 2
        padded_image = cv2.copyMakeBorder(gray_image, pad, pad, pad, pad, cv2.BORDER_CONSTANT)

        for i in range(pad, gray_image.shape[0] + pad):
            for j in range(pad, gray_image.shape[1] + pad):
                local_mean = np.mean(padded_image[i - pad:i + pad + 1, j - pad:j + pad + 1])
                local_stddev = np.std(padded_image[i - pad:i + pad + 1, j - pad:j + pad + 1])
                threshold = sauvola_threshold(local_mean, local_stddev)
                result_image[i - pad, j - pad] = 255 if gray_image[i - pad, j - pad] > threshold else 0

        # Save the binarized image using cv2.imwrite
        count=extract_number_from_string(img)
        suv_filename = f'suv_image{count}.png'
        suv_filepath = os.path.join(suv_images, suv_filename)
        cv2.imwrite(suv_filepath, result_image)



# su_binarization

In [13]:
import os
import cv2
from tqdm import tqdm
import numpy as np

def su_binarization(img, window_size=15, k=0.2, r=128):
    # Convert the image to grayscale if it's in color
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Pad the image to handle borders
    padded_img = cv2.copyMakeBorder(img, window_size // 2, window_size // 2, window_size // 2, window_size // 2, cv2.BORDER_CONSTANT, value=r)

    # Initialize output image
    binary_img = np.zeros_like(img)

    # Loop through each pixel of the input image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the local mean and standard deviation
            window = padded_img[i:i+window_size, j:j+window_size]
            local_mean = np.mean(window)
            local_std = np.std(window)

            # Calculate the threshold using SU formula
            threshold = local_mean * (1 + k * ((local_std / r) - 1))

            # Binarize the pixel
            if img[i, j] > threshold:
                binary_img[i, j] = 255
            else:
                binary_img[i, j] = 0

    return binary_img

def apply_su(src_uniform, su_images, window_size=15, k=0.2, r=128):
    # Create the output directory if it doesn't exist
    os.makedirs(su_images, exist_ok=True)

    # Loop through each image in the source directory
    for img_name in tqdm(os.listdir(src_uniform), desc="SU progress"):
        # Get the full path of the image
        img_path = os.path.join(src_uniform, img_name)

        # Read the image
        input_image = cv2.imread(img_path)

        # Check if the image is loaded successfully
        if input_image is None:
            print(f"Error: Could not open or find the image {img_name}.")
            continue

        # Binarize the image using SU binarization
        binary_image = su_binarization(input_image, window_size, k, r)

        # Save the binarized image
        count = extract_number_from_string(img_name)
        su_filename = f'su_image{count}.png'
        su_filepath = os.path.join(su_images, su_filename)
        cv2.imwrite(su_filepath, binary_image)

# Example usage:
# apply_su("input_images/", "output_su_images/", window_size=15, k=0.2)


In [ ]:
%ls

groundtruth/  jia_result/  otsu_result/     sr_sauvola_result/
howe_result/  original/    sauvola_result/  Traditional_Alg.ipynb


# sr_sauvola

In [ ]:
def sr_sauvola_folder(src_folder, sr_suv_images, window_size=15, k=0.2, r=128):


    os.makedirs(sr_suv_images, exist_ok=True)
    count = 1

    def sr_sauvola(img, window_size=15, k=0.2, r=128):
        pad = window_size // 2
        img = cv2.copyMakeBorder(img, pad, pad, pad, pad, cv2.BORDER_REPLICATE)

        binary_img = np.zeros_like(img)
        for i in range(pad, img.shape[0] - pad):
            for j in range(pad, img.shape[1] - pad):
                window = img[i - pad:i + pad + 1, j - pad:j + pad + 1]
                mean = np.mean(window)
                std = np.std(window)
                threshold = mean * (1 + k * (std / r - 1))  # SR Sauvola threshold
                binary_img[i, j] = 255 if img[i, j] > threshold else 0

        return binary_img[pad:-pad, pad:-pad]  # Remove padding

    for img_filename in tqdm(os.listdir(src_folder),desc="sr sauvola progress"):
        image = cv2.imread(os.path.join(src_folder, img_filename))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Apply SR Sauvola's Binarization
        binary_image = sr_sauvola(gray, window_size, k, r)
        count=extract_number_from_string(img_filename)
        sr_suv_filename = f'sr_suv_image{count}.png'
        sr_suv_filepath = os.path.join(sr_suv_images, sr_suv_filename)
        cv2.imwrite(sr_suv_filepath, binary_image)


extract number from string

In [5]:
def extract_number_from_string(input_string):
    # Define a regular expression pattern to match all consecutive digits
    pattern = r'\d'

    # Use re.findall() to find all matches in the string
    matches = re.findall(pattern, input_string)

    # If matches are found, return the list of all matches
    if matches:
        result= [match for match in matches]
        return "".join((result))
    else:
        return None  # or raise an exception, depending on your requirements

metrics

In [6]:
#metrics calculation
def calculate_DRD_k(ground_truth, segmented):
    # Assuming both ground_truth and segmented are grayscale images
    # Compute absolute intensity differences between corresponding pixels
    abs_diff = np.abs(ground_truth.astype(int) - segmented.astype(int))
    return abs_diff

def calculate_metrics(ground_truth, segmented):
    ground_truth = ground_truth.astype(bool)
    segmented = segmented.astype(bool)

    # True Positive, False Positive, False Negative
    TP = np.sum(np.logical_and(ground_truth, segmented))
    FP = np.sum(np.logical_and(~ground_truth, segmented))
    FN = np.sum(np.logical_and(ground_truth, ~segmented))

    # Precision, Recall
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0

    # F-measure
    f_measure = (2 * precision * recall) / (precision + recall) if precision + recall != 0 else 0

    # Pseudo F-measure
    pRecall = np.sum(ground_truth) / np.sum(ground_truth | segmented) if np.sum(ground_truth | segmented) != 0 else 0
    fps = (2 * pRecall * precision) / (pRecall + precision) if pRecall + precision != 0 else 0

    # Distance Reciprocal Distortion Metric (DRD)
    DRD_k = calculate_DRD_k(ground_truth, segmented)
    NUBN = np.sum(ground_truth)  # Assuming each non-zero pixel represents a uniform background region
    drd = np.sum(DRD_k) / NUBN if NUBN != 0 else 0

    # Peak Signal-to-Noise Ratio (PSNR)
    C = 255  # Assuming pixel values are in the range [0, 255]
    MSE = np.mean((ground_truth.astype(float) - segmented.astype(float))**2)
    psnr = 10 * np.log10(C**2 / MSE) if MSE != 0 else 0

    # Intersection over Union (IoU)
    intersection = np.logical_and(ground_truth, segmented)
    union = np.logical_or(ground_truth, segmented)
    iou = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0

    # Dice Coefficient
    dice = (2 * np.sum(intersection)) / (np.sum(ground_truth) + np.sum(segmented)) if (np.sum(ground_truth) + np.sum(segmented)) != 0 else 0

    return f_measure, fps, drd, psnr, iou, dice

def metrics(ground_test, binarized, metrics_output):
    for i in tqdm(range(len(ground_test)),desc="metrics progress"):
        f_measure, fps, drd, psnr, iou, dice = calculate_metrics(ground_test[i], binarized[i])
        metrics_output.append([f_measure, fps, drd, psnr, iou, dice])

    arr = np.array(metrics_output)
    res = np.mean(arr, axis=0)
    return res


reading binarized images

In [7]:
def read_binary_images(folder):
    images_str = input_sort(folder)[:]
    images = []
    for image in tqdm(images_str,desc="reading binarized images"):
        #print(image)
        img = cv2.imread(os.path.join(folder, image), cv2.IMREAD_GRAYSCALE)
        img = img / 255
        img = torch.Tensor(img).unsqueeze(0).detach().numpy()

        images.append(img)

    return images

# niblack

In [ ]:
def apply_niblack(src_uniform, niblack_images, window_size=15, k=0.2):
    # Read the image
    os.makedirs(niblack_images, exist_ok=True)

    for img in tqdm(os.listdir(src_uniform),desc="niblack progress"):
        img_path = os.path.join(src_uniform, img)

        input_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)


        # Check if the image is loaded successfully
        if input_image is None:
            print("Error: Could not open or find the image.")
            return

        # Set parameters for niBlackThreshold
        max_value = 255
        threshold_type = cv2.THRESH_BINARY
        block_size = 101  # Adjust according to your image
        k = -0.2  # Adjust according to your image
        binarization_method = cv2.ximgproc.BINARIZATION_NIBLACK

        # Apply niBlackThreshold
        thresholded = cv2.ximgproc.niBlackThreshold(input_image, max_value, threshold_type, block_size, k, binarization_method)

        # Save the image using cv2.imwrite
        count = extract_number_from_string(img)
        niblack_filename = f'niblack_image{count}.png'
        niblack_filepath = os.path.join(niblack_images, niblack_filename)

        cv2.imwrite(niblack_filepath, thresholded)


# testing

In [8]:


def test(orig_test,groundtruth):
    res = []

    ground_test=read_ground_images(groundtruth)

    '''binarization_methods = [
        ('Otsu', apply_otsu),
        ('niblac',apply_niblack)
        ('Sauvola', sauvola_binarization_folder),
        ('SR_Sauvola', sr_sauvola_folder),
        ('su_binarization',su_binarization_folder)

    ]'''
    binarization_methods = [
        ('su_binarization',apply_su),

        ]


    table_data = []
    i=0
    for bin_method, apply_func in tqdm(binarization_methods, desc="Binarization Progress"):
        # Applying the specific binarization method
        result_images_folder = f"{bin_method.lower()}_result"
        apply_func(orig_test, result_images_folder)
        print(f"{bin_method} images after binarization stored in '{result_images_folder}' folder")



        # Reading binarized images and calculating metrics
        binarized = read_binary_images(result_images_folder)
        metrics_result = metrics(ground_test, binarized, [])
        table_data.append([f"{bin_method}", *metrics_result])

    # Print the results in a tabular format
    print("\n****** Printing metrics table ********")
    headers = ["Model_Binarization", "F_measure", "FPS", "DRD", "PSNR", "IoU", "Dice"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))


input and sort

In [9]:
def input_sort(folder):
    name=os.listdir(folder)
    name.sort()
    return name

In [10]:
def read_images(folder):
    file=input_sort(folder)[:]
    store=[]
    for image in tqdm(file,desc="reading original images"):
        img=cv2.imread(os.path.join(folder,image))
        store.append(torch.Tensor(img).permute(2,0,1).detach().numpy())
    return store

In [11]:
def read_ground_images(folder):
    file=input_sort(folder)[:]
    ground=[]
    for image in tqdm(file,desc="reading groundtruth images"):
        img=cv2.imread(os.path.join(folder,image),cv2.IMREAD_GRAYSCALE)
        img=img/255
        ground.append(torch.Tensor(img).unsqueeze(0).detach().numpy())
    return ground

# main function

In [14]:
if __name__ == "__main__":
    original = "original"
    # original = input("Enter folder name of original images: ")
    ground_truth = "groundtruth"
    # ground_truth = input("Enter folder name of ground_truth images: ")
    test(original,ground_truth)

Binarization Progress:   0%|          | 0/1 [1:09:16<?, ?it/s]


KeyboardInterrupt: 

ImportError: cannot import name 'ximgproc' from 'cv2' (C:\Users\venuv\anaconda3\Lib\site-packages\cv2\__init__.py)